In [1]:
# !pip install SMOTE

In [2]:
# !pip install imbalanced-learn

In [3]:
# !pip install -U scikit-learn imbalanced-learn

In [36]:
pip install pyldavis

Note: you may need to restart the kernel to use updated packages.


In [38]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis.lda_model

In [5]:
from functions import *

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
train.describe()

In [ ]:
train.head(20)

In [ ]:
train.isnull().sum()

In [ ]:
# train = train.dropna()

In [ ]:
train.isnull().sum()

In [ ]:
train.duplicated().sum()

In [ ]:
train = train.drop_duplicates()

In [ ]:
train.shape

In [ ]:
class_counts = train['overall'].value_counts()

plt.bar(class_counts.index, class_counts.values)
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.title('Histogram of Sample Counts per Class')
plt.show()

In [ ]:
test.describe()

In [ ]:
test.head(20)

In [ ]:
test.isnull().sum()

In [ ]:
test.duplicated().sum()

In [ ]:
test.shape

In [6]:
train["Review"]= train["Review"].astype(str)


In [7]:
train["Review"] = train["Review"].apply(preprocess)

In [ ]:
train

In [ ]:
# train = train.head(8000).copy()

In [11]:
vectorizer = TfidfVectorizer()

In [ ]:
# vectorizer = CountVectorizer()

In [ ]:
X = vectorizer.fit_transform(train["Review"])

In [ ]:
y = train["overall"]
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,test_size=0.3, random_state=42)
print('Original class distribution:', Counter(y_train))


In [ ]:
undersampler = RandomUnderSampler(sampling_strategy={5: Counter(y_train)[4]}, random_state=42)
X_train_under, y_train_under = undersampler.fit_resample(X_train, y_train)

In [ ]:
print('Class distribution after undersampling:', Counter(y_train_under))


In [ ]:
smote = SMOTE()
X_train_smote, y_train_smote = smote.fit_resample(X_train_under, y_train_under)

In [ ]:
print('Class distribution after SMOTE:', Counter(y_train_smote))


In [ ]:
model = RandomForestClassifier()
model.fit(X_train_smote, y_train_smote)

In [ ]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))
y_train_pred = model.predict(X_train_smote)
print(classification_report(y_train_smote, y_train_pred))

In [8]:
high_rated = train[train['overall'] == 5]
low_rated = train[train['overall'] == 1]

In [9]:
high_rated['Review'] = high_rated['Review'].apply(preprocess)
low_rated['Review'] = low_rated['Review'].apply(preprocess)

/var/folders/ly/mbvbtlm97tv5q9v_g2g81z5h0000gn/T/ipykernel_65509/4093898252.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_rated['Review'] = high_rated['Review'].apply(preprocess)
/var/folders/ly/mbvbtlm97tv5q9v_g2g81z5h0000gn/T/ipykernel_65509/4093898252.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  low_rated['Review'] = low_rated['Review'].apply(preprocess)


In [21]:
high = vectorizer.fit_transform(high_rated['Review'])
low = vectorizer.fit_transform(low_rated['Review'])

In [24]:
# Number of topics
n_topics = 10

# Run LDA
lda_high = LatentDirichletAllocation(n_components=n_topics, random_state=0)
lda_high.fit(high)

lda_low = LatentDirichletAllocation(n_components=n_topics, random_state=0)
lda_low.fit(low)



LatentDirichletAllocation(random_state=0)

In [41]:
panel_high = pyLDAvis.lda_model.prepare(lda_high, high, vectorizer, mds='tsne')
pyLDAvis.display(panel_high)

AssertionError: Term frequencies and vocabulary are of different sizes, 40236 != 11760.

In [39]:
# Visualize low-rated topics
panel_low = pyLDAvis.lda_model.prepare(lda_low, low, vectorizer, mds='tsne')
pyLDAvis.display(panel_low)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pan